In [69]:
import requests
import pprint
import time
import openpyxl
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}
# url="https://m.weibo.cn/api/container/getIndex?uid=3591355593&luicode=10000011&lfid=1076033591355593&type=uid&value=3591355593&containerid=1076033591355593"
proxies={'HTTPS': '61.154.64.195:9999'}
#创建excel 表格
workbook = openpyxl.Workbook()
#使用一张数据表
sheet = workbook.active


def download_hot_child_comment(mid):
    hot_child_comment_url = 'https://m.weibo.cn/comments/hotFlowChild?cid='+mid+ '&max_id=0&max_id_type=0'
    hot_child_comment_res = requests.get(hot_child_comment_url,headers=headers,proxies=proxies)
    hot_child_data = hot_child_comment_res.json()
#     print('hot_child_data',hot_child_data)  
    hot_child_list=hot_child_data['data']
    for data in hot_child_list:
        text=data['text']
        mid = data['mid']
        user_name =data['user']['screen_name']
        sheet.append(['热评里的评论',user_name,mid,text])
        
  
def download_hot_comment(mid): 
    sub_comment_url ='https://m.weibo.cn/comments/hotflow?id='+str(mid)+'&mid=' + mid + '&max_id_type=0'
    sub_res = requests.get(sub_comment_url,headers,proxies=proxies)
    sub_data = sub_res.json()   
    data_list = sub_data['data']['data']
    for data in data_list:
#         print('data',data)
        text = data['text']
        mid = data['mid']
        if 'more_info_users' in data:
#             screen_name = data['more_info_users'][0]['screen_name'] 视频的错误代码，这个取的不是用户名了
#             print('sub_commit:',[screen_name, mid, text])
            screen_name = data['user']['screen_name']
            sheet.append(['热评',screen_name,mid,text])            
            #热评下面继续有热评的子评论
            try:
                download_hot_child_comment(mid)
            except:
                pass
        
def download_one_page(comment_url):
    res=requests.get(url=comment_url,headers=headers,proxies=proxies)
    comment_data = res.json() 
    #只有字典格式可以使用json接口数据请求
    #字典数据    
    #格式化打印漂亮的输出
#     pprint.pprint(comment_data)   
    cards = comment_data['data']['cards']
#     print('cards:',cards)
    # 一页里面的十条数据
    for card in cards:
        if card.get('mblog'):
            m_blog=card.get('mblog')
            text = m_blog.get('text')
            mid = m_blog.get('mid')
            screen_name= m_blog.get('user').get('screen_name')
#             print([screen_name,mid,text])
            #每条数据都是有热评
            #请求一条数据的热捧            
            sheet.append(['微博正文',screen_name,mid,text])
            download_hot_comment(mid)
        break
            
            
def main():
    for page in range(1,2):
#         time.sleep(1)  #防止爬的快被封
        comment_url ='https://m.weibo.cn/api/container/getIndex?uid=3591355593&t=0&luicode=10000011&lfid=100103type%3D1%26q%3Dwuyifan&type=uid&value=3591355593&containerid=1076033591355593&page='+str(page)
        #print(comment_url)
        download_one_page(comment_url)

if __name__ == "__main__":
    main()

workbook.save('吴先生.xlsx')
print('over')

cards: [{'card_type': 9, 'itemid': '1076033591355593_-_4660583661568436', 'scheme': 'https://m.weibo.cn/status/Kpq3A6A1m?mblogid=Kpq3A6A1m&luicode=10000011&lfid=1076033591355593', 'mblog': {'visible': {'type': 0, 'list_id': 0}, 'created_at': 'Mon Jul 19 08:06:36 +0800 2021', 'id': '4660583661568436', 'mid': '4660583661568436', 'can_edit': False, 'version': 198, 'show_additional_indication': 36, 'text': '之前没有回应是因为不想干扰司法程序的推进，但没想到我的沉默却纵容了造谣者的变本加厉，我已忍无可忍！本人只在2020年12月5日的朋友聚会中见过都女士一次，没有灌酒、没有收手机、更没有她描述的各种“细节”，当天聚会人很多，都会作证！我很抱歉打扰到大家。我申明，从来没有过什么“选妃”！没有 ...<a href="/status/4660583661568436">全文</a>', 'textLength': 397, 'source': '', 'favorited': False, 'pic_ids': [], 'pic_types': '', 'is_paid': False, 'mblog_vip_type': 0, 'user': {'id': 3591355593, 'screen_name': 'Mr_凡先生', 'profile_image_url': 'https://tvax2.sinaimg.cn/crop.0.0.670.670.180/003V2XEdly8gq2rsbgj6fj60im0inmyz02.jpg?KID=imgbed,tva&Expires=1626854409&ssig=RisZ5dJ3Tn', 'profile_url': 'https://m.weibo.cn/u/3591355593?uid=3591355593&luic

over
